In [7]:
from django.db import connection, reset_queries

from cinema.models import Award, Person, Movie, Nomination
from cinema.choices import MOVIE_GENRES
from cinema.utils import print_sql
from cinema.populate_database import populate_database

In [8]:
populate_database()

# Reset queries so they won't impact future output
reset_queries()

# Começando pelo básico

## Filter

https://books.agiliq.com/projects/django-orm-cookbook/en/latest/join.html

In [17]:
queryset = Movie.objects.filter(genre=MOVIE_GENRES.action)

print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."synopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."is_available_on_netflix",
       "cinema_movie"."imdb_rate",
       "cinema_movie"."director_id",
       "cinema_movie"."budget"
FROM "cinema_movie"
WHERE "cinema_movie"."genre" = action



## Exclude

In [9]:
queryset = Movie.objects.exclude(imdb_rate__gt=6.5)

print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."synopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."director_id",
       "cinema_movie"."is_available_on_netflix",
       "cinema_movie"."imdb_rate"
FROM "cinema_movie"
WHERE NOT ("cinema_movie"."imdb_rate" > 6.5
           AND "cinema_movie"."imdb_rate" IS NOT NULL)



## Filtrando "através" de tabelas
https://docs.djangoproject.com/en/4.0/topics/db/queries/#lookups-that-span-relationships

In [10]:
queryset = Movie.objects.filter(director__name="Tim Burton")
# str(queryset.query)
print_sql(queryset)

SELECT "cinema_movie"."id",
       "cinema_movie"."title",
       "cinema_movie"."synopsis",
       "cinema_movie"."genre",
       "cinema_movie"."release_date",
       "cinema_movie"."director_id",
       "cinema_movie"."is_available_on_netflix",
       "cinema_movie"."imdb_rate"
FROM "cinema_movie"
INNER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")
WHERE "cinema_person"."name" = Tim Burton



## Cuidado com alguns detalhes

https://stackoverflow.com/questions/8164675/chaining-multiple-filter-in-django-is-this-a-bug/28253623#28253623

Doc do Django: https://docs.djangoproject.com/en/3.2/topics/db/queries/#spanning-multi-valued-relationships

In [11]:
queryset_1 = Person.objects.filter(
    movies_directed__title__icontains="batman", movies_directed__release_date__year__gt=2000
)

print_sql(queryset_1)

queryset_1

SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date"
FROM "cinema_person"
INNER JOIN "cinema_movie" ON ("cinema_person"."id" = "cinema_movie"."director_id")
WHERE ("cinema_movie"."release_date" > 2000-12-31
       AND UPPER("cinema_movie"."title"::text) LIKE UPPER(%batman%))



<QuerySet [<Person: Christopher Nolan>]>

In [7]:
queryset_2 = Person.objects.filter(movies_directed__title__icontains="batman").filter(
    movies_directed__release_date__year__gt=2000
).distinct()

print_sql(queryset_2)

queryset_2

SELECT DISTINCT "cinema_person"."id",
                "cinema_person"."name",
                "cinema_person"."birth_date",
                "cinema_person"."death_date"
FROM "cinema_person"
INNER JOIN "cinema_movie" ON ("cinema_person"."id" = "cinema_movie"."director_id")
INNER JOIN "cinema_movie" T3 ON ("cinema_person"."id" = T3."director_id")
WHERE (UPPER("cinema_movie"."title"::text) LIKE UPPER(%batman%)
       AND T3."release_date" > 2000-12-31)



<QuerySet [<Person: Tim Burton>, <Person: Christopher Nolan>]>

queryset_1 vai filtrar por diretores que já dirigiram filmes com "batman" no título **e** lançados depois de 2000.

Já queryset_2 vai filtrar por diretores que tanto já dirigiram filmes com "batman" no título quanto filmes lançados depois de 2000.

Isso acontece por causa da forma como o Django processa o filter; tudo dentro de uma única chamada filter() é aplicado simultaneamente para filtrar os itens que correspondem a todos esses requisitos. Já chamadas filter() sucessivas restringem ainda mais o conjunto de objetos, mas para relações multivaloradas, elas se aplicam a qualquer objeto vinculado ao modelo primário, não necessariamente àqueles objetos que foram selecionados por uma chamada filter() anterior.

**Obs**: O mesmo não acontece com .exclude(). Tudo dentro de uma única chamada exclude() não se refere necessariamente ao mesmo item. Para conseguir dar um exclude em objetos que vão dar match com as condições dadas, é preciso realizar duas consultas.

Exemplo da doc (adaptar):
Blog.objects.exclude(
    entry__in=Entry.objects.filter(
        headline__contains='Lennon',
        pub_date__year=2008,
    ),
)

# Problema de N+1

O ORM do Django faz com que a gente interaja com o banco de dados relacional de forma que parece natural pro paradigma de orientação a objetos. Então se no nosso código a gente faz algo como isso aqui, onde temos o queryset de um model e depois iteramos pelo queryset acessando um atributo do objeto e um atributo de um objeto referenciado pelo objeto, isso parece ser certo.

In [12]:
reset_queries()

queryset = Movie.objects.all()

for i, movie in enumerate(queryset, start=1):
    print(f"{i}. >{movie.title}< was directed by {movie.director.name}")

1. >Deep Water< was directed by Adrian Lyne
2. >Batman< was directed by Tim Burton
3. >Batman Returns< was directed by Tim Burton
4. >Batman Begins< was directed by Christopher Nolan
5. >The Dark Knight< was directed by Christopher Nolan
6. >The Dark Knight Rises< was directed by Christopher Nolan
7. >Inglourious Basterds< was directed by Quentin Tarantino
8. >Once Upon a Time in... Hollywood< was directed by Quentin Tarantino
9. >Knives Out< was directed by Rian Johnson
10. >Shutter Island< was directed by Martin Scorsese
11. >Hide and Seek< was directed by John Polson
12. >13 Going on 30< was directed by Gary Winick
13. >Don't Look Up< was directed by Adam McKay
14. >Big Fish< was directed by Tim Burton


Mas, na verdade, para cada iteração do for, a gente tá acessando o banco de dados duas vezes (fazendo duas queries). A gente pode conferir isso usando o `connection.queries`:

In [13]:
connection.queries

[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."synopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."director_id", "cinema_movie"."is_available_on_netflix", "cinema_movie"."imdb_rate" FROM "cinema_movie"',
  'time': '0.001'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" WHERE "cinema_person"."id" = 1128 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" WHERE "cinema_person"."id" = 1129 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" WHERE "cinema_person"."id" = 1129 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date"

## select_related

O Django sabe do problema e ele dá uma solução para isso, que é o `select_related`. E é usado da seguinte forma: se você vai iterar por um queryset de um model e você precisa acessar um atributo de um campo desse model que é uma referência a uma outra tabela (um Foreign Key field), você chama o select_related depois do objects e passa o nome do campo.

In [14]:
reset_queries()

queryset = Movie.objects.select_related("director").all()

for i, movie in enumerate(queryset, start=1):
    print(f"{i}. >{movie.title}< was directed by {movie.director.name}")

1. >Deep Water< was directed by Adrian Lyne
2. >Big Fish< was directed by Tim Burton
3. >Batman Returns< was directed by Tim Burton
4. >Batman< was directed by Tim Burton
5. >The Dark Knight Rises< was directed by Christopher Nolan
6. >The Dark Knight< was directed by Christopher Nolan
7. >Batman Begins< was directed by Christopher Nolan
8. >Once Upon a Time in... Hollywood< was directed by Quentin Tarantino
9. >Inglourious Basterds< was directed by Quentin Tarantino
10. >Knives Out< was directed by Rian Johnson
11. >Shutter Island< was directed by Martin Scorsese
12. >Hide and Seek< was directed by John Polson
13. >13 Going on 30< was directed by Gary Winick
14. >Don't Look Up< was directed by Adam McKay


O que está sendo feito ao usar o select_related é que, no SQL usado, o SELECT vai conter os campos do model do queryset e os campos daquele campo que você vai acessar. Como pode ser visto aqui:

In [15]:
connection.queries

[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."synopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."director_id", "cinema_movie"."is_available_on_netflix", "cinema_movie"."imdb_rate", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_movie" LEFT OUTER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")',
  'time': '0.002'}]

In [5]:
reset_queries()

queryset = Movie.objects.select_related("director").only("title", "director").all()

for movie in queryset:
    print(f"{movie.title} was directed by {movie.director.name}")

connection.queries

Deep Water was directed by Adrian Lyne


[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."director_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_movie" LEFT OUTER JOIN "cinema_person" ON ("cinema_movie"."director_id" = "cinema_person"."id")',
  'time': '0.001'}]

Se você precisar "limpar" a lista dos campos relacionados que foram adicionados por chamadas do select_related, basta usar `select_related(None)`.

## prefetch_related

Works for FK relationships, but also for M2M relation ships. However, the optimization is made by Python and not the SQL.

In [6]:
reset_queries()

queryset = Movie.objects.all()

for movie in queryset:
    print(f"{movie.title} cast: \n")
    
    for person in movie.cast.all():
        print(f"- {person.name}\n")
        
connection.queries

Deep Water cast: 

- Ana de Armas

- Ben Affleck

- Jacob Elordi

- Finn Wittrock

- Rachel Blanchard



[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.000'},
 {'sql': 'SELECT "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" INNER JOIN "cinema_movie_cast" ON ("cinema_person"."id" = "cinema_movie_cast"."person_id") WHERE "cinema_movie_cast"."movie_id" = 3',
  'time': '0.001'}]

In [7]:
reset_queries()

queryset = Movie.objects.prefetch_related("cast").all()

for movie in queryset:
    print(f"{movie.title} cast: \n")
    
    for person in movie.cast.all():
        print(f"- {person.name}\n")
        
connection.queries

Deep Water cast: 

- Ana de Armas

- Ben Affleck

- Jacob Elordi

- Finn Wittrock

- Rachel Blanchard



[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."sinopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id" FROM "cinema_movie"',
  'time': '0.000'},
 {'sql': 'SELECT ("cinema_movie_cast"."movie_id") AS "_prefetch_related_val_movie_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" INNER JOIN "cinema_movie_cast" ON ("cinema_person"."id" = "cinema_movie_cast"."person_id") WHERE "cinema_movie_cast"."movie_id" IN (3)',
  'time': '0.001'}]

In [6]:
reset_queries()

Movie.objects.prefetch_related("cast").all()

# connection.queries

<QuerySet [<Movie: Deep Water>, <Movie: Batman>, <Movie: Batman Returns>, <Movie: Batman Begins>, <Movie: The Dark Knight>, <Movie: The Dark Knight Rises>, <Movie: Inglourious Basterds>, <Movie: Once Upon a Time in... Hollywood>, <Movie: Knives Out>, <Movie: Shutter Island>, <Movie: Hide and Seek>, <Movie: 13 Going on 30>, <Movie: Don't Look Up>, <Movie: Big Fish>]>

In [7]:
connection.queries

[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."synopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."is_available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id", "cinema_movie"."budget" FROM "cinema_movie" LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT ("cinema_movie_cast"."movie_id") AS "_prefetch_related_val_movie_id", "cinema_person"."id", "cinema_person"."name", "cinema_person"."birth_date", "cinema_person"."death_date" FROM "cinema_person" INNER JOIN "cinema_movie_cast" ON ("cinema_person"."id" = "cinema_movie_cast"."person_id") WHERE "cinema_movie_cast"."movie_id" IN (80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93)',
  'time': '0.001'}]

In [10]:
reset_queries()

Movie.objects.filter(cast__name__istartswith="g")

<QuerySet [<Movie: Batman Begins>, <Movie: The Dark Knight>, <Movie: The Dark Knight Rises>]>

In [11]:
connection.queries

[{'sql': 'SELECT "cinema_movie"."id", "cinema_movie"."title", "cinema_movie"."synopsis", "cinema_movie"."genre", "cinema_movie"."release_date", "cinema_movie"."is_available_on_netflix", "cinema_movie"."imdb_rate", "cinema_movie"."director_id", "cinema_movie"."budget" FROM "cinema_movie" INNER JOIN "cinema_movie_cast" ON ("cinema_movie"."id" = "cinema_movie_cast"."movie_id") INNER JOIN "cinema_person" ON ("cinema_movie_cast"."person_id" = "cinema_person"."id") WHERE UPPER("cinema_person"."name"::text) LIKE UPPER(\'g%\') LIMIT 21',
  'time': '0.001'}]

Saber usar select_related e prefetch_related é algo muito importante para as pessoas que usam Django, porque é fácil cair no problema de N+1 ou problemas similares e ao olhar pro código você pode não sabe que ali tem um problema de performance.

## .count() e .exists()

São duas coisas diferentes que as pessoas às vezes "confundem" e que podem acabar prejudicando a performance da sua aplicação se você não se ligar.

Acontece às vezes da gente usar `queryset.count() > 0` em vez de `queryset.exists()` e não perceber que isso pode ter um impacto negativo.
Acontece que o count vai contar todas as linhas daquela tabela, enquanto o exists apenas checa se existe pelo menos uma linha naquela tabela. Se você tá lidando com um queryset que tem muitos objetos, o count se torna um problema.

`.count()` a gente deve usar apenas se a gente quiser saber o número de objetos num queryset.
`.exists()` a gente deve usar para saber se aquele queryset não é vazio.

(Fazer comparação de tempo entre um e outro)

In [23]:
reset_queries()

Movie.objects.filter(is_available_on_netflix=True).count()

connection.queries

[{'sql': 'SELECT COUNT(*) AS "__count" FROM "cinema_movie" WHERE "cinema_movie"."is_available_on_netflix"',
  'time': '0.001'}]

In [22]:
reset_queries()

Movie.objects.filter(is_available_on_netflix=True).exists()

connection.queries

[{'sql': 'SELECT (1) AS "a" FROM "cinema_movie" WHERE "cinema_movie"."is_available_on_netflix" LIMIT 1',
  'time': '0.000'}]

## Retornar apenas o necessário!

`only`, `defer`, `values` e `values_list` vão te ajudar com isso.

In [13]:
queryset = Person.objects.all()

print_sql(queryset)

SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date"
FROM "cinema_person"



In [5]:
print_sql(queryset.only("name"))

SELECT "cinema_person"."id",
       "cinema_person"."name"
FROM "cinema_person"



In [6]:
print_sql(queryset.defer("name"))

SELECT "cinema_person"."id",
       "cinema_person"."birth_date",
       "cinema_person"."death_date"
FROM "cinema_person"



In [14]:
print_sql(queryset.values_list("name"))
queryset.values_list("name")

SELECT "cinema_person"."name"
FROM "cinema_person"



<QuerySet [('Ana de Armas',), ('Ben Affleck',), ('Jacob Elordi',), ('Finn Wittrock',), ('Rachel Blanchard',), ('Michael Keaton',), ('Michelle Pfeiffer',), ('Danny DeVito',), ('Christopher Walken',), ('Michael Gough',), ('Pat Hingle',), ('Jack Nicholson',), ('Kim Basinger',), ('Christian Bale',), ('Cillian Murphy',), ('Michael Caine',), ('Katie Holmes',), ('Liam Neeson',), ('Gary Oldman',), ('Morgan Freeman',), '...(remaining elements truncated)...']>

In [15]:
print_sql(queryset.values("name"))
queryset.values("name")

SELECT "cinema_person"."name"
FROM "cinema_person"



<QuerySet [{'name': 'Ana de Armas'}, {'name': 'Ben Affleck'}, {'name': 'Jacob Elordi'}, {'name': 'Finn Wittrock'}, {'name': 'Rachel Blanchard'}, {'name': 'Michael Keaton'}, {'name': 'Michelle Pfeiffer'}, {'name': 'Danny DeVito'}, {'name': 'Christopher Walken'}, {'name': 'Michael Gough'}, {'name': 'Pat Hingle'}, {'name': 'Jack Nicholson'}, {'name': 'Kim Basinger'}, {'name': 'Christian Bale'}, {'name': 'Cillian Murphy'}, {'name': 'Michael Caine'}, {'name': 'Katie Holmes'}, {'name': 'Liam Neeson'}, {'name': 'Gary Oldman'}, {'name': 'Morgan Freeman'}, '...(remaining elements truncated)...']>

## Annotate

Dá mais poder ao desenvolvedor.

### Count

Exemplo: você quer uma lista ordenada a partir de uma contagem específica.

In [16]:
from django.db.models import Count

queryset = Person.objects.annotate(
    total_awards_won=Count("nominations")
).order_by("-total_awards_won")

print_sql(queryset)

SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date",
       COUNT("cinema_personnomination"."id") AS "total_awards_won"
FROM "cinema_person"
LEFT OUTER JOIN "cinema_personnomination" ON ("cinema_person"."id" = "cinema_personnomination"."person_id")
GROUP BY "cinema_person"."id"
ORDER BY "total_awards_won" DESC



### Count + filter

In [17]:
from django.db.models import Count, Q

queryset = Person.objects.annotate(
    total_awards_won=Count("nominations", filter=Q(nominations__is_winner=True))
).order_by("-total_awards_won")

print_sql(queryset)

list(queryset)

SELECT "cinema_person"."id",
       "cinema_person"."name",
       "cinema_person"."birth_date",
       "cinema_person"."death_date",
       COUNT("cinema_personnomination"."id") FILTER (
                                                     WHERE "cinema_personnomination"."is_winner") AS "total_awards_won"
FROM "cinema_person"
LEFT OUTER JOIN "cinema_personnomination" ON ("cinema_person"."id" = "cinema_personnomination"."person_id")
GROUP BY "cinema_person"."id"
ORDER BY "total_awards_won" DESC



[<Person: Heath Ledger>,
 <Person: Brad Pitt>,
 <Person: Christoph Waltz>,
 <Person: Dakota Fanning>,
 <Person: Adam McKay>,
 <Person: Jennifer Garner>,
 <Person: Michael Keaton>,
 <Person: Mélanie Laurent>,
 <Person: Judy Greer>,
 <Person: Ariana Grande>,
 <Person: Michael Gough>,
 <Person: Daniel Craig>,
 <Person: Jennifer Lawrence>,
 <Person: Albert Finney>,
 <Person: Austin Butler>,
 <Person: Chris Evans>,
 <Person: Mark Ruffalo>,
 <Person: Margaret Qualley>,
 <Person: Marion Cotillard>,
 <Person: Tom Hardy>,
 <Person: Gary Winick>,
 <Person: Timothée Chalamet>,
 <Person: Christa B. Allen>,
 <Person: Patricia Clarkson>,
 <Person: Matthew McGrory>,
 <Person: Aaron Eckhart>,
 <Person: Christopher Walken>,
 <Person: Cate Blanchett>,
 <Person: Eli Roth>,
 <Person: Leonardo Dicaprio>,
 <Person: Jack Nicholson>,
 <Person: Robert De Niro>,
 <Person: Christopher Plummer>,
 <Person: Ewan McGregor>,
 <Person: Morgan Freeman>,
 <Person: Billy Crudup>,
 <Person: Rian Johnson>,
 <Person: Ben Ki

### Subquery e OuterRef

In [15]:
from django.db.models import OuterRef, Subquery, Q

q = Award.objects.annotate(
        most_nominated_movie_title=Subquery(
            Movie.objects.annotate(
                nominations_count=Count('nominations', filter=Q(nominations__award_id=OuterRef('id')))
            ).order_by('-nominations_count').values("title")[:1]
        )
)

print_sql(q)

q.values_list("name", "most_nominated_movie_title")

SELECT "cinema_award"."id",
       "cinema_award"."name",
       "cinema_award"."popular_name",
       "cinema_award"."first_awarded_year",

  (SELECT U0."title"
   FROM "cinema_movie" U0
   LEFT OUTER JOIN "cinema_movienomination" U1 ON (U0."id" = U1."movie_id")
   GROUP BY U0."id"
   ORDER BY COUNT(U1."id") FILTER (
                                   WHERE (U1."award_id" = "cinema_award"."id")) DESC
   LIMIT 1) AS "most_nominated_movie_title"
FROM "cinema_award"



<QuerySet [('Academy Awards', 'The Dark Knight'), ('British Academy Film Awards', 'The Dark Knight'), ('American Comedy Awards', '13 Going on 30'), ('MTV Movie + TV Awards', '13 Going on 30')]>